### Applied Data Science Capstone - Assignment (Week 3)

Importing the necessary libraries and scraping the wiki page into a pandas dataframe.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [2]:
# Reference: https://www.youtube.com/watch?v=D8T1epVAx-4
df = df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [3]:
df[df['Neighbourhood']=='Not assigned']['Neighbourhood'] = df[df['Neighbourhood']=='Not assigned']['Borough']
df.head()

D:\Installations\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combining rows with common neighbourhoods, adding longitude latitude informations and creating a new dataframe named Processed_df.

In [10]:
# Read the Geospatial data to a dataframe
lon_lat_df = pd.read_csv('Geospatial_Coordinates.csv')

Postcodes = df['Postcode'].unique()
Processed_df = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])

for i in range(len(Postcodes)):
    Latitude = float(lon_lat_df[lon_lat_df['Postal Code']==Postcodes[i]]['Latitude'])
    Longitude = float(lon_lat_df[lon_lat_df['Postal Code']==Postcodes[i]]['Longitude'])
    #combining common neighbourhoods
    df_temp = df[df['Postcode']==Postcodes[i]]
    Neighbourhoods = ''
    for (idx, row) in df_temp.iterrows():
        current_neighborhood = row.loc['Neighbourhood']
        current_Borough = row.loc['Borough']
        if len(Neighbourhoods)==0:
            Neighbourhoods = current_neighborhood
        else:
            Neighbourhoods = Neighbourhoods + ',' + current_neighborhood
    Processed_df = Processed_df.append({'Postcode' : Postcodes[i] ,
                                        'Borough' : current_Borough, 
                                        'Neighbourhood' : Neighbourhoods,
                                        'Latitude': Latitude,
                                       'Longitude': Longitude}, 
                                       ignore_index=True)
Processed_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


Displyaing the shape (number of rows and columns) in the processed dataframe

In [5]:
Processed_df.shape 

(103, 3)

In [13]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

In [16]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Use geopy library to get the latitude and longitude values of Toronto City.

In [17]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [22]:
# create map of Toronto using latitude and longitude values
map_Toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for Latitude, Longitude, Borough, Neighbourhood in zip(Processed_df['Latitude'], Processed_df['Longitude'], Processed_df['Borough'], Processed_df['Neighbourhood']):
    label = '{}, {}'.format(Processed_df, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto